<a href="https://colab.research.google.com/github/kk25gb/AIdea-project-file/blob/Siamese-network/Indexer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 建立索引清單

In [2]:
import json
import csv
from urllib.request import urlopen
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import tqdm

In [20]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/main/stage2/vectorize_data.json'
response = urlopen(url)
data = json.loads(response.read())
known_assay_labels = list(data.keys())
df = pd.DataFrame(data).T

In [5]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/similarity_list_stage_2.json'
response = urlopen(url)
similarity_list = json.loads(response.read())

In [ ]:
similarity_list

In [6]:
url = 'https://raw.githubusercontent.com/kk25gb/AIdea-project-file/Siamese-network/stage2/column_filter.json'
response = urlopen(url)
column_filter = json.loads(response.read())

In [31]:
object_list = df[column_filter[list(column_filter.keys())[index]]].loc[list(similarity_list.keys())[index]]
object_list.iloc[:].tolist()

[6, 14, 2, 3]

In [70]:
index = 0
filter1 = df[column_filter[list(column_filter.keys())[index]]].loc[similarity_list[list(similarity_list.keys())[index]]]
filter1
# 

,370,578,583,584
269,3,7,1,1
889,7,13,5,3
660,1,5,4,1
390,5,6,4,1
315,7,5,4,1
1001,6,14,2,3
451,3,4,3,1
1123,4,8,1,1
399,5,7,1,1
19,3,7,1,2


In [71]:
index_list = column_filter[list(column_filter.keys())[index]]
index_list

[370, 578, 583, 584]

In [72]:
for key, value in zip(filter1.iloc[:,:].index,filter1.iloc[:,:].values):
  print(key, value.tolist())

269 [3, 7, 1, 1]
889 [7, 13, 5, 3]
660 [1, 5, 4, 1]
390 [5, 6, 4, 1]
315 [7, 5, 4, 1]
1001 [6, 14, 2, 3]
451 [3, 4, 3, 1]
1123 [4, 8, 1, 1]
399 [5, 7, 1, 1]
19 [3, 7, 1, 2]
1019 [1, 7, 4, 1]
1034 [3, 5, 1, 1]
1040 [4, 5, 1, 1]
1091 [6, 10, 9, 1]
1022 [4, 8, 1, 5]
996 [7, 14, 2, 3]


In [76]:
i = '999'
index_object = df[column_filter[i]].loc[i].tolist()
print("Object:",i,index_object)
compare_object = df[column_filter[i]].loc[similarity_list[i]]
compare_object

Object: 999 [2, 2, 27, 1, 10, 10]


,35,109,395,561,575,581
999,2,2,27,1,10,10


In [69]:
len(compare_object.iloc[:,:].values)>1

False

# 進行迭代分析

In [ ]:
for i in list(column_filter.keys()):
  index_object = df[column_filter[i]].loc[i].tolist()
  print("Object:",i,index_object)
  compare_object = df[column_filter[i]].loc[similarity_list[i]]
  if len(compare_object.iloc[:,:].index)>1:
    for key, value in zip(compare_object.iloc[:,:].index,compare_object.iloc[:,:].values):
      if key!=i:print(key, value.tolist())
  # break
  print('----------------------')

In [ ]:
filter1 = df[column_filter[list(column_filter.keys())[index]]].loc[similarity_list[list(similarity_list.keys())[index]]]